<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore')
import requests
from bs4 import BeautifulSoup
import plotly.express as px

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''
select count(*)
from public.vacancies
'''

In [5]:
# Получаем DataFrame с помощью read_sql_query
vacanciesCount = pd.read_sql_query(query_3_1, connection)

print('Количество вакансий:')
display(vacanciesCount)

Количество вакансий:


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f'''
select count(*)
from public.employers
'''

In [7]:
# результат запроса
employersCount = pd.read_sql_query(query_3_2, connection)

print('Количество работодателей:')
display(employersCount)

Количество работодателей:


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''
select count(*)
from public.areas
'''

In [9]:
# результат запроса
areasCount = pd.read_sql_query(query_3_3, connection)

print('Количество регионов:')
display(areasCount)

Количество регионов:


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''
select count(*)
from public.industries
'''

In [11]:
# результат запроса
industriesCount = pd.read_sql_query(query_3_4, connection)

print('Количество сфер деятельности:')
display(industriesCount)

Количество сфер деятельности:


,count
0,294


***

### Вывод: база данных содержит большое количество информации, предоставляя информацию о 50 тысячах вакансий от более чем 20 тысяч работодателей из различных районов, размещающих вакансии практически по 300 различным областям деятельности. 

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
# текст запроса
query_4_1 = f'''
select
    a.name area,
    count(*) cnt
from
    public.vacancies v -- Соединяем с таблицей регионов, для группировки по их названиям
    join public.areas a on v.area_id = a.id
group by a.name
order by count(*) desc
'''

In [13]:
# результат запроса
vacanciesByAreas = pd.read_sql_query(query_4_1, connection)

print('Количество вакансий по регионам')
display(vacanciesByAreas.head(5))

Количество вакансий по регионам


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
# текст запроса
query_4_2 =f'''
select
    count(*)
from public.vacancies v
where v.salary_from is not NULL -- Фильтруем по ненулевым значениям хотя бы в 1 поле
or v.salary_to is not NULL
'''

In [15]:
# результат запроса
oneNotNullSalary = pd.read_sql_query(query_4_2, connection)

print('Количество вакансий хотя бы с одним полем зарплаты:')
display(oneNotNullSalary)

Количество вакансий хотя бы с одним полем зарплаты:


,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# текст запроса
query_4_3 = f'''
select
    round(avg(v.salary_from)) lower_salary_limit,
    round(avg(v.salary_to)) upper_salary_limit
from public.vacancies v
'''

In [17]:
# результат запроса
salatyFork = pd.read_sql_query(query_4_3, connection)

print('Границы зарплатной вилки')
display(salatyFork)

Границы зарплатной вилки


,lower_salary_limit,upper_salary_limit
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# текст запроса
query_4_4 = f'''
select
    v.schedule,
    v.employment,
    count(*) -- Количество вакансий для различных пар трудоустройства и графика
from public.vacancies v
group by v.schedule, v.employment
order by count(*) desc
'''

In [19]:
# результат запроса
scheduleWithEmployment = pd.read_sql_query(query_4_4, connection)

print('Количество различных сочетаний рабочего графика и трудоустройства')
display(scheduleWithEmployment)

Количество различных сочетаний рабочего графика и трудоустройства


,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
# текст запроса
query_4_5 = f'''
select
    v.experience "Требуемый опыт работы",
    count(*) -- Количество различных вариантов опыта работы
from public.vacancies v
group by v.experience
order by count(*)
'''

In [21]:
# результат запроса
requiredExperience = pd.read_sql_query(query_4_5, connection)

display(requiredExperience)

,Требуемый опыт работы,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

### Вывод: в ходе анализа вакансий выявлены наиболее необходимые работодателям соискатели в зависимости от региона(больше всего вакансий в г.Москва), требуемой занятости(более всего работодатели ищут человека на полный рабочей день с полной занятостью) и опыта работы(больше всего вакансий для соискателей с небольшим опытом работы - от 1 до 3 лет). Также установленны средние границы зарплаты, предлагаемой работадателями, представленными в данной БД.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
# текст запроса
query_5_1 = f'''
select
    e.name company_name,
    count(*) -- Количество вакансий от различных работодателей
from
    public.vacancies v -- Присоединям таблицу работодателей для группировки вакансий по названию компании
    join public.employers e on v.employer_id = e.id
group by e.name
order by count(*) desc
limit 5
'''

In [23]:
# результат запроса
popularEmployers = pd.read_sql_query(query_5_1, connection)

print('Работодатели с наибольшим количеством вакансий')
display(popularEmployers)

Работодатели с наибольшим количеством вакансий


,company_name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
# текст запроса
query_5_2 = f'''
select
    a.name region_name,
    count(distinct e.id) employers_cnt, -- Количество работодателей
    count(v.id) vacancies_cnt -- Количество  вакансий для каждого региона
from
    public.areas a -- Присоединяем таблицы для получения названий регионов и компаний
    left join public.employers e on e.area = a.id
    left join public.vacancies v on (v.employer_id = e.id) and (a.id = v.area_id) -- Присоединяем таблицу вакансий с учетом указанного работодателя
group by a.name
order by count(v.id), count(distinct e.id) desc
limit 1
'''

In [25]:
# результат запроса
areasDestribution = pd.read_sql_query(query_5_2, connection)

display(areasDestribution)

,region_name,employers_cnt,vacancies_cnt
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
# текст запроса
query_5_3 = f'''
select
    e.name Employer,
    count(distinct v.area_id) areas_cnt
from
    public.employers e
    join public.vacancies v on v.employer_id = e.id
group by e.name
order by count(distinct v.area_id) desc
'''

In [27]:
# результат запроса
employersAreasCnt = pd.read_sql_query(query_5_3, connection)

print('Количество регионов, в которых компании предоставляют вакансии')
display(employersAreasCnt)

Количество регионов, в которых компании предоставляют вакансии


,employer,areas_cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [28]:
# текст запроса
query_5_4 = f'''
select
    count(*)
from
    public.employers e -- Получаем сферы деятельности для всех имеющихся работодателей
    left join public.employers_industries ei on e.id = ei.employer_id
where ei.industry_id is NULL -- Учитываем только работодателей, у которых не указана сфера деятельности
'''

In [29]:
# результат запроса
NaNIndustry = pd.read_sql_query(query_5_4, connection)

print('Количество работадателей без сферы деятельности:')
display(NaNIndustry)

Количество работадателей без сферы деятельности:


,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [30]:
# текст запроса
query_5_5 = f'''
select
    e.name employer_name
from
    public.employers e -- Получаем сферы деятельности, у кого они имеются
    join public.employers_industries ei on e.id = ei.employer_id
group by e.name
having count(ei.industry_id) = 4 -- Учитываем только компании с 4 сферами деятельности
order by 1 -- Сортируем по алфавиту(столбец employer_name)
offset 2 limit 1 -- Получаем 3 строку
'''

In [31]:
# результат запроса
fourIndustryEmployers = pd.read_sql_query(query_5_5, connection)

print('Третья компания по алфавиту с 4 сферами деятельности:')
display(fourIndustryEmployers)

Третья компания по алфавиту с 4 сферами деятельности:


,employer_name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [32]:
# текст запроса
query_5_6 = f'''
select
    count(distinct e.id)
from
    public.employers e -- Получаем работодателей, которые указали сферу деятельности, и название деятельности
    join public.employers_industries ei on e.id = ei.employer_id
    join public.industries i on ei.industry_id = i.id
where i.name = 'Разработка программного обеспечения' 
'''

In [33]:
# результат запроса
programmers = pd.read_sql_query(query_5_6, connection)

print('Количество работодателей по разработке программного обеспечения:')
display(programmers)

Количество работодателей по разработке программного обеспечения:


,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [34]:
# код для получения списка городов-миллионников

# Ссылка на сайт с городами-миллионниками
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
soup = BeautifulSoup(requests.get(url).text, 'html.parser')
table = soup.select('table')[1] # Получаем таблицу с городами-миллионниками(html-вид)

millionCities = table.find_all('a') # Инфромация о городах содержится в тегах <a>

for i, tag in enumerate(millionCities): # Переводим тэги в текст, который в них содержится
    millionCities[i] = tag.text 

# В таблице с городами содержатся приписки к ним в тех же тегах, удаляем их
millionCities = tuple(filter(lambda x: x[0].isupper(), millionCities))

millionCities

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [35]:
# текст запроса
query_5_7 = f'''
select
    a.name,
    count(*) cnt -- Получаем количество вакансий
from
    public.vacancies v 
    join public.areas a on v.area_id = a.id
    join public.employers e on v.employer_id = e.id
where e.name = 'Яндекс' -- в компании "Яндекс"
and a.name in {millionCities} -- которые относятся к городам-миллионникам
group by a.name

union all -- Объединяем таблицы

select
    'Total',
    count(*) -- Получаем количество вакансий по всем регионам-миллионникам в России для той же компании
from
    public.vacancies v
    join public.areas a on v.area_id = a.id
    join public.employers e on v.employer_id = e.id
where e.name = 'Яндекс'
and a.name in {millionCities}

order by 2 -- Сортируем по возрастанию количества
'''

In [36]:
# результат запроса
YandexCnt = pd.read_sql_query(query_5_7, connection)

print('Количество вакансий компании "Яндекс" в городах-миллионниках')
display(YandexCnt)

Количество вакансий компании "Яндекс" в городах-миллионниках


,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

### Вывод: больше всего вакансий предоставляет компания "Яндекс", предоставляя их в 181 регионе России и во всех городах-миллионниках. Также многие работодатели не указывают сферу деятельности(более 8 тысяч), при этом для соискателей по разработке программного обеспечения предложено 3,5 тысячи вакансий, что может говорить о нехватке кадров в сфере IT 

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [37]:
# текст запроса
query_6_1 = f'''
select
    count(*)
from public.vacancies v
where lower(v.name) like '%data%'
or lower(v.name) like '%данн%'
'''

In [38]:
# результат запроса
dataVacancies = pd.read_sql_query(query_6_1, connection)

print('Количество вакансий, относящихся к данным:')
display(dataVacancies)

Количество вакансий, относящихся к данным:


,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [39]:
# текст запроса
query_6_2 = f'''
select
    count(*)
from
    public.vacancies v
where
(lower(v.name) like '%data scientist%' 
    or lower(v.name) like '%data science%'
    or v.name like '%ML%'
    or lower(v.name) like '%machine learning%'
    or lower(v.name) like '%машинн%обучен%')
and v.name not like '%HTML%' -- Условия отношения названия вакансии к професии Data Scientist
and (lower(v.name) like '%junior%' or v.experience = 'Нет опыта' or v.employment = 'Стажировка') -- Условие начинающего специалиста
'''

In [40]:
# результат запроса
juniorDataVacancies = pd.read_sql_query(query_6_2, connection)

print('Количество вакансий для начинающих дата-сайентистов:')
display(juniorDataVacancies)

Количество вакансий для начинающих дата-сайентистов:


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [41]:
# текст запроса
query_6_3 = f'''
select
    count(*)
from
    public.vacancies v
where
(lower(v.name) like '%data scientist%'
    or lower(v.name) like '%data science%'
    or v.name like '%ML%'
    or lower(v.name) like '%machine learning%'
    or lower(v.name) like '%машинн%обучен%')
and v.name not like '%HTML%' -- Условия отношения названия вакансии к професии Data Scientist
and (v.key_skills like '%SQL%' or lower(v.key_skills) like '%postgres%') -- Условие по ключевому навыку
'''

In [42]:
# результат запроса
sqlNeedDS = pd.read_sql_query(query_6_3, connection)

print('Количество вакансий DS, в которых необходим SQL:')
display(sqlNeedDS)

Количество вакансий DS, в которых необходим SQL:


,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [43]:
# текст запроса
query_6_4 = f'''
select
    count(*)
from
    public.vacancies v
where
(lower(v.name) like '%data scientist%'
    or lower(v.name) like '%data science%'
    or v.name like '%ML%'
    or lower(v.name) like '%machine learning%'
    or lower(v.name) like '%машинн%обучен%')
and v.name not like '%HTML%' -- Условия отношения названия вакансии к професcии Data Scientist
and lower(v.key_skills) like '%python%' -- Условие по ключевому навыку
'''

In [44]:
# результат запроса
pythonNeedDS = pd.read_sql_query(query_6_4, connection)

print('Вакансии DS, где требуют знание Python:')
display(pythonNeedDS)

Вакансии DS, где требуют знание Python:


,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [45]:
# текст запроса
query_6_5 = f'''
select -- Среднее количество ключевых навыком, разделенных через '\t'
    round(avg(array_length(regexp_split_to_array(v.key_skills, '\t'), 1)), 2) mean
from
    public.vacancies v
where
(lower(v.name) like '%data scientist%'
    or lower(v.name) like '%data science%'
    or v.name like '%ML%'
    or lower(v.name) like '%machine learning%'
    or lower(v.name) like '%машинн%обучен%')
and v.name not like '%HTML%' -- Условия отношения названия вакансии к професcии Data Scientist
'''

In [46]:
# результат запроса
keySkillsCnt = pd.read_sql_query(query_6_5, connection)

print('Среднее количество навыков для дата-сайентиста:')
display(keySkillsCnt)

Среднее количество навыков для дата-сайентиста:


,mean
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [47]:
# текст запроса
query_6_6 = f'''
select
    v.experience, 
    -- Группируем поле с зарплатой по вышеизложенным условиям
    round(avg((coalesce(v.salary_from, v.salary_to) + coalesce(v.salary_to, v.salary_from)) / 2)) mean_salary
from
    public.vacancies v
where
(lower(v.name) like '%data scientist%'
    or lower(v.name) like '%data science%'
    or v.name like '%ML%'
    or lower(v.name) like '%machine learning%'
    or lower(v.name) like '%машинн%обучен%')
and v.name not like '%HTML%' -- Условия отношения названия вакансии к профеcсии Data Scientist
and (v.salary_from is not NULL or v.salary_to is not NULL) 
group by v.experience
'''

In [48]:
# результат запроса
meanSalaryDS = pd.read_sql_query(query_6_6, connection)

print('Средняя зарплата дата-сайентиста в зависимости от опыта работы:')
display(meanSalaryDS)

Средняя зарплата дата-сайентиста в зависимости от опыта работы:


,experience,mean_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

### Вывод: для направления Data Science представленно суммарно почти 500 вакансий, при этом доля поиска начинающих специалистов составляет 20% от этого числа, также любопытно, что в среднем требуется знание более 6 различных ключевых навыков(языки программирования и различные методы работы с данными). Помимо этого видно, что при наличии опыта работы всего в 3 года, заработная плата специалиста в среднем увеличивается в 3 раза, что можно встретить далеко не в каждой сфере трудовой деятельности.  

# Общий вывод по проекту

### По итогу исследования данных выявили зависимости в данных, распределения количества вакансий от таких показателей, как опыт работы, занятость, рабочий график(оказалось, что более всего требуются специалисты с небольшим опытом работы(скорее всего молодые - для успешного обучения и продвижения компании засчет молодых амбициозных работников),на полную занятость(может быть связано с большей эффективностью работу непосредственно в офисе или с необходимостью работы на каком-либо объекте в течение дня)).Так же установленно, что компания "Яндекс" предоставляет рабочие места в наибольшем объёме по сравнению с рстальными компаниями, а также в самом большом количестве регионов(это может быть связано с тем, что "Яндекс" - крупнейшая технологическая компания в России и ведущая в Европе, следовательно им нужно много работников в различных районах нашей(и не только) страны). И важное наблюдение - более половины работодателей не указывают зарплату при размещении вакансии, что является помехой для работы рекомендательной системы _hh.ru_, следовательно данные нуждаются в заполнении с учетом взаимосвязей других признаков(например в зависимости от региона, занятости, рабочего графика и др.).

## Доп исследования

Зарплата DS специалиста в зависимости от региона

In [50]:
# Получаем список стран
# Ссылка на сайт с странами
url = 'https://ru.wikipedia.org/wiki/Список_государств'
soup = BeautifulSoup(requests.get(url).text, 'html.parser')
table = soup.select('table')[0] # Получаем таблицу со странами(html-вид)

countries = table.find_all('a') # Инфромация о странах содержится в тегах <a>

for i, tag in enumerate(countries): # Переводим тэги в текст, который в них содержится
    countries[i] = tag.text 

# В таблице со странами содержатся пустые строки в тех же тегах, удаляем их
countries = tuple(filter(None, countries))

In [51]:
# текст запроса
query_add = f'''
select
    a.name area_name,
    round(avg((coalesce(v.salary_from, v.salary_to) + coalesce(v.salary_to, v.salary_from)) / 2)) mean_salary
from
    public.vacancies v
    join public.areas a on v.area_id = a.id
where
(lower(v.name) like '%data scientist%'
    or lower(v.name) like '%data science%'
    or v.name like '%ML%'
    or lower(v.name) like '%machine learning%'
    or lower(v.name) like '%машинн%обучен%')
and v.name not like '%HTML%' -- Условия отношения названия вакансии к професcии Data Scientist
and (v.salary_from is not NULL or v.salary_to is not NULL) -- Оставляем строки с хотя бы 1 заполненным полем зарплаты
and (a.name in {countries} or a.name in {millionCities}) -- Регион относится к другим странам или городам-миллионникам России
group by a.name
order by 2 desc -- Сортируем по средней зарплате
'''

In [52]:
# результат запроса
areaDepSalary = pd.read_sql_query(query_add, connection)

# Строим столбчатую диаграмму для визуализации полученных данных
fig = px.bar(
    areaDepSalary,
    x='area_name',
    y='mean_salary',
    width=1000,
    height=800
)

fig.update_layout(
    title='Зависимость региона работы от зарплаты DS',
    xaxis_title='Страна(регион) проживания',
    yaxis_title='Средняя зарплата'
)

fig.show()

### Вывод: у дата-сайентистов наибольшая зарплата не в России, при этом в России она наибольшая в крупнейших городах(Москва, Санкт-Петербург и Новосибирск). Это може быть связано с тем, что в других странах используются другие валюты, что при переводе в рубли дает соответствующие показатели, что касается России - зарплата наибольшая в указанных городах скорее всего потому, что эти города являются наиболее развивающимися и прогрессивными, поэтому там специалисты получают больше.

Количество вакансий DS в зависимости от региона

In [53]:
# текст запроса
query_add_2 = f'''
select
    a.name area_name,
    count(*)
from
    public.vacancies v
    join public.areas a on v.area_id = a.id
where
(lower(v.name) like '%data scientist%'
    or lower(v.name) like '%data science%'
    or v.name like '%ML%'
    or lower(v.name) like '%machine learning%'
    or lower(v.name) like '%машинн%обучен%')
and v.name not like '%HTML%' -- Условия отношения названия вакансии к професcии Data Scientist
and (v.salary_from is not NULL or v.salary_to is not NULL) -- Оставляем строки с хотя бы 1 заполненным полем зарплаты
and (a.name in {countries} or a.name in {millionCities}) -- Регион относится к другим странам или городам-миллионникам России
group by a.name
order by 2 desc
'''

In [54]:
# результат запроса
areaDepVacanciesCnt = pd.read_sql_query(query_add_2, connection)

# Строим столбчатую диаграмму для визуализации полученных данных
fig = px.bar(
    areaDepVacanciesCnt,
    x='area_name',
    y='count',
    width=1000,
    height=800
)

fig.update_layout(
    title='Зависимость региона работы от количества вакансий DS(с указанием зарплаты)',
    xaxis_title='Страна(регион) проживания',
    yaxis_title='Количество вакансий'
)

fig.show()

### Вывод: из диаграммы видно, что наибольшее количество вакансий для дата-сайентистов относятся к региону Москва, что, если учитывать предыдущий график, делает этот город лучшим выбором для DS специалиста в вопросах поиска работы. Это может быть связано с большим количеством компаний в данном регионе, так как Москва - активно развивающийся город(+ с большой численностью начеления, даже большей, чем во всех представленных в данных странах, не считая РФ). И соответственно чем больше компаний, тем больше требуется специалистов.

Соотношение работодателей, указывающих зарплатную вилку в вакансии

In [56]:
# текст запроса
query_add_3 = f'''
select
    'with_salary' salary,
    count(*) vacancies_cnt
from
    public.vacancies v
where (v.salary_from is not NULL or v.salary_to is not NULL) -- Хотя бы одно непустое поле зарплаты

union all -- Объединяем полученные таблицы 

select
    'without_salary',
    count(*)
from
    public.vacancies v
where (v.salary_from is NULL and v.salary_to is NULL) -- Зарплата не заполненна ни в одном поле
'''

In [57]:
# результат запроса
isSalary = pd.read_sql_query(query_add_3, connection)
connection.close() # Закрываем соединение

# Строим круговую диаграмму для визуализации полученных данных
fig = px.pie(
    isSalary,
    names='salary',
    values='vacancies_cnt',
    title='Диаграмма распределения вакансий по указанию зарплаты(хотя бы одной границы)',
    width=800,
    height=600
)


fig.show()

### Вывод: из распределения видно, что более половины работодателей не указывают ни одной границы необходимой зарплаты для соискателя. Это может быть связано с тем, что зарплата сильно зависит от уровня пришедшего специалиста, или если работодатель не определился с суммой(характерно для новых специальностей в компании). Тем не менее, главная опасность - невозможность работы рекомендательной системы сервиса, которая подбирает для компании подходящих работников(та же проблема наблюдается в анализе резюме в проекте 1, но со стороны соискателей), следовательно данные нуждаются в обработке для прогнозирования зарплаты по тем или иным специальностям, сферам деятельности и т.д.